# watsonx.aiのLLMでLangChainを使ってPDFの内容をQ&Aをする

https://qiita.com/nishikyon/items/5054209089fc632981f8/

# 必要なライブラリのインストール

In [ ]:
!pip install -U ibm-watson-machine-learning
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install unstructured
!pip install sentence_transformers
!pip install flask-sqlalchemy

**インストール終了後、一旦カーネルを再起動してください**

#  (オプション)LangChainで使えるLLMの確認
`LLAMA_2_70B_CHAT`が表示されたか確認してください。
表示されていない場合は`ibm-watson-machine-learning`のバージョンが古いです。

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

# LangChainで使えるLLMの取得

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com", # watsonx.aiのAuthentication用のエンドポイントのURL
    "apikey": "<APIキー>"
}
project_id = "<PROJECT ID>"

# 使用するLLMのパラメータ
generate_params = {
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# モデルの初期化
model = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT, #使用するLLM名
    credentials=credentials,
    params=generate_params,
    project_id=project_id
)

# LangChainで使うllm
custom_llm = WatsonxLLM(model=model)



# そのまま実行

In [ ]:
result=custom_llm("IBM Db2 on Cloudの特徴は?")
print(result)

# PDFの取得

In [ ]:
!wget https://files.speakerdeck.com/presentations/cc34f85fe9b5467d8782a41e5fa39b78/Dojo_Db2RESTAPI_20230727_%E9%85%8D%E5%B8%83%E7%94%A8.pdf

# PDFLoaderの作成

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Dojo_Db2RESTAPI_20230727_配布用.pdf") #ダウンロードしたPDFを指定

# indexの作成

`ImportError: cannot import name 'URL' from 'sqlalchemy'`というエラーがでたら、一旦カーネルをリスタートして、やり直してください。

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma

index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings()).from_loaders([loader])

# プロプントを考えずに聞いてみる

In [ ]:
query ="IBM TechXchange Japanとは。"
answer = index.query(llm=custom_llm, question=query)
print(answer)

# 試しに絶対にPDFにない簡単な質問をする

In [ ]:
query ="日本の首都は?"
answer = index.query(llm=custom_llm, question=query)
print(answer)

# retrieverの作成

In [ ]:
retriever = index.vectorstore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={'score_threshold': 0.5}
)

# プロンプトの作成

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """[INST] <<SYS>>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
<</SYS>>
Question: {question}
Answer in Japanese:[/INST]"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# RetrievalQAの作成

In [ ]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=custom_llm, chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs=chain_type_kwargs, 
    return_source_documents=True
)

# 質問してみます

In [ ]:
query = "IBM TechXchange Japanとは?"
result = qa({"query": query})
print(result["result"])
print("-------------------")
print(result["source_documents"])

In [ ]:
query = "日本の首都は?"
result = qa({"query": query})
print(result["result"])
print("-------------------")
print(result["source_documents"])

In [ ]:
query = "Db2 on Cloud REST APIを使うには、まず何をしますか?"
result = qa({"query": query})
print(result["result"])
print("-------------------")
print(result["source_documents"])

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=custom_llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs=chain_type_kwargs
)

query = "Db2 on Cloud REST APIを使うには、まず何をしますか?" 
answer = qa.run(query)
print(answer)